# Séries génératrices

Nous nous intéressons ici aux calculs de séries génératrices des mots tassés et des sous-ensembles des mots tassés (irréductibles, rouges-irréductibles, bleus-irréductibles, doubles-irréductibles).

Nous ajoutons ici une deuxième graduation qui correspond à la valeur maximale. Par exemple le terme ```(24*r^4 + 36*r^3 + 14*r^2 + r)*z^4``` signifie que pour les mots tassés de taille $4$: il y en a $24$ dont le maximum est $4$, $36$ dont le maximum est $3$, $14$ dont le maximum est $2$ et $1$ seul dont le maximum est $1$.

### Équations des théorèmes algébriques sur les séries

In [1]:
from packed_words import *

PR.<r> = PolynomialRing(QQ)
PSR.<z> = PowerSeriesRing(PR)
# z : degré = longueur
# r : lettre max

N = 5 # À changer pour avoir plus de termes

In [2]:
# Dimensions de WQSym+ (ou mots tassés non vides)
A = sum([z^n*sum(r^k*stirling_number2(n, k) * factorial(k) 
                 for k in range(n+1)) 
         for n in range(1,6)]) + O(z^N)
A

r*z + (2*r^2 + r)*z^2 + (6*r^3 + 6*r^2 + r)*z^3 + (24*r^4 + 36*r^3 + 14*r^2 + r)*z^4 + O(z^5)

In [3]:
A == sum(z^n * sum(r^(max(p)) 
                   for p in PackedWords(n))
         for n in range(1,N)) + O(z^N)

True

In [4]:
# Dimensions de Prim(WQSym) (ou mots tassés irréductibles)
P = A / (1 + A)
P

r*z + (r^2 + r)*z^2 + (3*r^3 + 4*r^2 + r)*z^3 + (13*r^4 + 23*r^3 + 11*r^2 + r)*z^4 + O(z^5)

In [5]:
P == sum(z^n * sum(r^(max(p)) 
                   for p in PackedWords(n) if p.is_irreducible())
         for n in range(1,5)) + O(z^N)

True

In [6]:
# Dimensions de TPrim(WQSym) (ou mots tassés rouges-irréductibles 
#                                         ou bleus-irréductibles)
T = A / (1 + A) ^ 2
T

r*z + r*z^2 + (r^3 + 2*r^2 + r)*z^3 + (6*r^4 + 13*r^3 + 8*r^2 + r)*z^4 + O(z^5)

In [7]:
T == sum(z^n * sum(r^(max(p)) 
                   for p in PackedWords(n) if p.is_irreducible("red"))
         for n in range(1,N)) + O(z^N)

True

Si nous regardons la série $\mathcal{A}/(1 + \mathcal{A})^3$ sans le paramètre $r$, tous les termes sont positifs,  nous pouvons donc supposer qu'il existe un sous-ensemble de mots tassés qui engendre librement tous les mots tassés en tant qu'algèbre bigraft.

In [8]:
A(z,1) / (1 + A(z,1))^3

z + z^3 + 14*z^4 + O(z^5)

Cependant, quand nous rajoutons le paramètre $r$, nous remarquons un terme négatif.

In [9]:
A / (1 + A)^3

r*z + (-r^2 + r)*z^2 + r*z^3 + (2*r^4 + 6*r^3 + 5*r^2 + r)*z^4 + O(z^5)

Ce terme négatif implique qu'il n'existe pas de sous ensemble de mots tassés qui engendre librement tous les mots tassés en tant qu'algèbre bigraft, comme nous l'avions supposé plus haut.

Voici la série des mots tassés à la fois rouges-irréductibles et bleus-irréductibles.

In [10]:
# Mots tassés à la fois rouges-irréductibles 
#                    et bleus-irréductibles
I = A / (1 + A)^3 + z*r*P
I

r*z + r*z^2 + (r^3 + r^2 + r)*z^3 + (5*r^4 + 10*r^3 + 6*r^2 + r)*z^4 + O(z^5)

In [23]:
I == sum(z^n * sum(r^(max(p)) 
                   for p in PackedWords(n) 
                   if p.is_irreducible("bicolored"))
         for n in range(1,N)) + O(z^N)

True

### Pompage pour obtenir les séries des forêts d'arbres bleus

Nous utilisons ici la grammaire suivante des forêts d'arbres bleus:
- Forest $\to$ [] | Tree,Forest
- Tree   $\to$ Node,Forest
- Node   $\to$ $1^\circ$ | $1^\bullet$,Tree | $i^\alpha$,Tree$^*$ (avec $2 \leq i \leq weight(Tree^*)$)

In [12]:
def pump_forest(Tree, Forest):
    return 1 + Tree * Forest

def pump_ske(Node, Forest):
    return Node * Forest

def pump_blue_node(BTree):
    res = r*z # correspond au mot 1
    res+= z * BTree 
    # correspond à l'ajout d'un 1 à la fin sans augmentation
    res+= (r + 1)*z * r^2 * (BTree/r).derivative(r) 
    # correspond à un noeud étiqueté avec i != 1
    return res

In [13]:
BForest = 1 + O(z)
BTree = 0 + O(z)
BNode = 0 + O(z)

In [14]:
for _ in range(N-1):
    BNode = pump_blue_node(BTree)
    BTree = pump_ske(BNode,BForest)
    BForest = pump_forest(BTree,BForest)
print(BNode)
print(BTree)
print(BForest)

r*z + r*z^2 + (r^3 + 2*r^2 + r)*z^3 + (6*r^4 + 13*r^3 + 8*r^2 + r)*z^4 + O(z^5)
r*z + (r^2 + r)*z^2 + (3*r^3 + 4*r^2 + r)*z^3 + (13*r^4 + 23*r^3 + 11*r^2 + r)*z^4 + O(z^5)
1 + r*z + (2*r^2 + r)*z^2 + (6*r^3 + 6*r^2 + r)*z^3 + (24*r^4 + 36*r^3 + 14*r^2 + r)*z^4 + O(z^5)


Voici plus de détails pour le passage de la grammaire à la fonction pour la dernière étape:

$i^\alpha$,Tree$^*$ (avec $2 \leq i \leq weight(Tree^*)$) qui devient ```(r + 1)*z * r^2 * (BTree/r).derivative(r) ```

In [15]:
BTree

r*z + (r^2 + r)*z^2 + (3*r^3 + 4*r^2 + r)*z^3 + (13*r^4 + 23*r^3 + 11*r^2 + r)*z^4 + O(z^5)

In [16]:
max1 = r*((BTree/(r))(z,0))
max1 # correspond aux mots dont le max est 1

r*z + r*z^2 + r*z^3 + r*z^4 + O(z^5)

In [17]:
max2 = r^2*(((BTree-max1)/r^2)(z,0))
max2 # correspond aux mots dont le max est 2

r^2*z^2 + 4*r^2*z^3 + 11*r^2*z^4 + O(z^5)

In [18]:
max3 = r^3*(((BTree-max1-max2)/r^3)(z,0))
max3 # correspond aux mots dont le max est 3

3*r^3*z^3 + 23*r^3*z^4 + O(z^5)

```BTree - max1``` correspond à l'ensemble des arbres pouvant être étiquetés par $2^\alpha$

```BTree - max1 - max2``` correspond à l'ensemble des arbres pouvant être étiquetés par $3^\alpha$

```BTree - max1 - max2 - max3``` correspond à l'ensemble des arbres pouvant être étiquetés par $4^\alpha$

[...]

La somme de tous ces ensembles correspond donc aux arbres pouvant être étiquetés par $i^\alpha$.

In [19]:
(BTree-max1) + (BTree-max1-max2) + (BTree-max1-max2-max3)

r^2*z^2 + (6*r^3 + 4*r^2)*z^3 + (39*r^4 + 46*r^3 + 11*r^2)*z^4 + O(z^5)

In [20]:
BTree

r*z + (r^2 + r)*z^2 + (3*r^3 + 4*r^2 + r)*z^3 + (13*r^4 + 23*r^3 + 11*r^2 + r)*z^4 + O(z^5)

Nous remarquons que pour une puissance de $z$ donnée, le coefficient de $r^2$ est compté $1$ fois, le coefficient de $r^3$ est compté $2$ fois, le coefficient de $r^4$ est compté $3$ fois et ainsi de suite. La formule pour les arbres pouvant être étiquetés par $i^\alpha$ est donc la suivante:

In [21]:
r^2*(BTree/r).derivative(r)

r^2*z^2 + (6*r^3 + 4*r^2)*z^3 + (39*r^4 + 46*r^3 + 11*r^2)*z^4 + O(z^5)

Lorsqu'un arbre de cet ensemble est étiqueté par $i^\alpha$ on ajoute 1 à la longueur du mot et le ```r + 1``` correspond à la valeur d'$\alpha$ (on ajoute un à la valeur max si $\alpha = \circ$, zéro si $\alpha = \bullet$). 

Finalement nous avons bien ``` (r + 1)*z * r^2 * (BTree/r).derivative(r) ```

Pour conclure, nous obtenons bien les mêmes séries que plus haut.

In [22]:
print(T - BNode)
print(P - BTree)
print(A + 1 - BForest)

O(z^5)
O(z^5)
O(z^5)


Il est possible d'appliquer un raisonnement similaire pour les forêts d'arbres rouges.